In [136]:
%matplotlib inline

import numpy as np
import pandas as pd

import librosa.display
import librosa

from glob import glob

import ffmpeg

import os

from pydub import AudioSegment
from pydub.utils import make_chunks

import re

In [160]:
def decompose_files(data_dir, audio_files):
    for j in range(len(audio_files)):
        myaudio = AudioSegment.from_file(data_dir + '{}'.format(os.listdir(data_dir)[j])) 
        chunk_length_ms = 10000
        chunks = make_chunks(myaudio, chunk_length_ms)

        for i, chunk in enumerate(chunks):
            chunk_name = "{}{}.wav".format(os.listdir(data_dir)[j], i)
            print("exporting", chunk_name)
            chunk.export('./sounds2/{}'.format(chunk_name), format="mp3")

In [161]:
decompose_files('./sounds/', glob('./sounds' + '/*'))

exporting ducha_5.mp324.wav0.wav
exporting hairdryer.mp319.wav0.wav
exporting cena.wav11.wav0.wav
exporting blender.mp33.wav0.wav
exporting shaver.mp333.wav0.wav
exporting microwave_2.mp327.wav0.wav
exporting dryer.mp345.wav0.wav
exporting dryer_2.mp329.wav0.wav
exporting faucet_2.mp35.wav0.wav
exporting alarm_2.mp322.wav0.wav
exporting airconditioner.mp30.wav0.wav
exporting washing_machine_2.mp342.wav0.wav
exporting doorbell_2.mp322.wav0.wav
exporting ducha_3.mp38.wav0.wav
exporting cellphone.mp341.wav0.wav
exporting comp_type.mp38.wav0.wav
exporting vacuum_2.mp37.wav0.wav
exporting cena.wav13.wav0.wav
exporting silence.mp331.wav0.wav


In [162]:
def get_features(data_dir, audio_files):
    momentos = {'ducha': 'Ducha', 'cena': 'Cena', 'washing': 'Lavadora', 'vacuum': 'Aspiradora', 
                'shaver': 'Afeitadora', 'hairdryer': 'Secador_pelo', 'airconditioner': 'Aire_acondicionado', 
                'cellphone': 'Telefono', 'comp': 'Tecleo', 'silence': 'Silencio', 'dryer': 'Secadora', 
                'blender': 'Licuadora', 'doorbell': 'Timbre', 'alarm': 'Alarma', 'faucet': 'Grifo', 
                'microwave': 'Microondas'}
    features = []
    for i in range(len(audio_files)):
        y, sr = librosa.load(audio_files[i])
        name = os.listdir(data_dir)[i]
        pattern = "[._][\w]+"
        name = re.sub(pattern, '', name)
        if name in momentos.keys():
            momento = momentos.get(name)
        else:
            momento = 'Otro'
        mfcc = np.ndarray.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20))
        scem = np.ndarray.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
        scom = np.ndarray.mean(librosa.feature.spectral_contrast(S=np.abs(librosa.stft(y)), sr=sr))
        srom = np.ndarray.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
        sbwm = np.ndarray.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
        tempo = librosa.beat.tempo(onset_envelope=librosa.onset.onset_strength(y=y, sr=sr, hop_length=512), sr=sr, hop_length=512)[0]
        rmse = np.ndarray.mean(librosa.feature.rms(y=y))
        features.append([mfcc, scem, scom, srom, sbwm, tempo, rmse, momento])
    return pd.DataFrame(features, columns=['mfcc', 'scem','scom', 'srom','sbwm', 'tempo', 'rmse', 'momento'])

In [163]:
data = get_features('./sounds2/', glob('./sounds2' + '/*.wav'))
data.head()

,mfcc,scem,scom,srom,sbwm,tempo,rmse,momento
0,-5.617919,4822.729698,17.012424,8490.626926,2922.254333,129.199219,0.056099,Grifo
1,-2.063152,452.804643,17.915530,811.142272,535.833205,129.199219,0.084744,Aire_acondicionado
2,-25.202837,1554.105964,27.932041,3095.535342,1372.209905,151.999081,0.035863,Cena
3,4.669409,669.271532,16.381503,727.032881,1716.868434,117.453835,0.143639,Secadora
4,-1.508489,1186.695657,18.275804,2155.318754,2039.238339,107.666016,0.149006,Secadora


In [164]:
data

,mfcc,scem,scom,srom,sbwm,tempo,rmse,momento
0,-5.617919,4822.729698,17.012424,8490.626926,2922.254333,129.199219,0.056099,Grifo
1,-2.063152,452.804643,17.915530,811.142272,535.833205,129.199219,0.084744,Aire_acondicionado
2,-25.202837,1554.105964,27.932041,3095.535342,1372.209905,151.999081,0.035863,Cena
3,4.669409,669.271532,16.381503,727.032881,1716.868434,117.453835,0.143639,Secadora
4,-1.508489,1186.695657,18.275804,2155.318754,2039.238339,107.666016,0.149006,Secadora
5,-3.035361,2508.270707,25.810619,4608.405235,2341.296911,117.453835,0.086121,Afeitadora
6,-12.056254,2727.620419,18.929312,5360.468591,2403.742428,123.046875,0.131954,Secador_pelo
7,-26.344146,1630.421174,26.129081,3218.989043,1388.219723,117.453835,0.032983,Cena
8,-2.924622,2876.000263,19.457674,6356.741453,2784.884735,117.453835,0.074781,Licuadora
9,-10.544885,3399.908268,22.133242,7362.332047,2706.150931,117.453835,0.156983,Alarma


In [165]:
data.shape

(19, 8)

In [166]:
data.dtypes

mfcc       float64
scem       float64
scom       float64
srom       float64
sbwm       float64
tempo      float64
rmse       float64
momento     object
dtype: object

In [167]:
data.momento.value_counts()

Ducha                 2
Cena                  2
Secadora              2
Lavadora              1
Secador_pelo          1
Afeitadora            1
Aire_acondicionado    1
Alarma                1
Tecleo                1
Telefono              1
Microondas            1
Licuadora             1
Grifo                 1
Timbre                1
Aspiradora            1
Silencio              1
Name: momento, dtype: int64

In [168]:
data.to_csv('data_prueba.csv', index=False)